In [ ]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

if hf_token:
    login(token=hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("HF_TOKEN not found in Colab Secrets. Please set it up.")
    raise ValueError("HF_TOKEN not found. Please configure it in Colab Secrets.")

Successfully logged in to Hugging Face!


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pickle
from time import time
from tqdm import tqdm
import pandas as pd

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

try:
    TOKENIZER = AutoTokenizer.from_pretrained(MODEL_ID)
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    exit()

try:
    MODEL = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.bfloat16, # 建議為 8B 模型添加，以節省記憶體
        device_map="auto", # 或者直接 .to(DEVICE) 如果 VRAM 足夠
    )
    # 如果 device_map="auto" 可能會將模型分片，此時 input_ids.to(model.device) 中的 model.device 可能不直接是 DEVICE
    # 一個更穩健的方式是 input_ids.to(MODEL.device) 或者在 generate 時模型和輸入在同一個主要設備上
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

def load_pickle(path):
    try:
        with open(path, 'rb') as file:
            file = pickle.load(file)
        return file
    except Exception as e:
        print(f"Loading pickle {path} faild: {e}")
    return None
def save_pickle(file, path):
    try:
        with open(path,'wb') as f:
            pickle.dump(file, f, protocol=pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print(f"save pickle {path} faild: {e}")

def load_prompt(file_path: str) -> str:
    """
    Load a prompt template from a file and replace `{N}` with the given integer.

    Args:
        file_path (str): The path to the prompt.txt file.


    Returns:
        str: The formatted prompt string with `{N}` replaced.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            prompt = file.read()
        return prompt
    except FileNotFoundError:
        raise FileNotFoundError(f"File not found: {file_path}")
    except Exception as e:
        raise RuntimeError(f"An error occurred while loading the prompt: {e}")
def articleID_to_product_type_name(article_df:pd.DataFrame, article_ids:list)->list[str]:
    """
    Parameters:
    - article_df: pd.Dataframe, must contain with cols ['article_id','product_type_name'].
    - article_ids: list of int, contain article ids of the user.

    Return:
    - category: list of string.
    """
    article_df['article_id'] = article_df['article_id'].astype(int)
    article_df['product_type_name'] = article_df['product_type_name'].astype(str)
    categroy = article_df[article_df['article_id'].isin(article_ids)]['product_type_name']
    return categroy.to_list()

def inference(system_content: str, user_content: str):

  messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
  ]
  try:
    input_ids = TOKENIZER.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(MODEL.device)
  except Exception as e:
      print(f"Error applying chat template: {e}")
      exit()

  terminators = [
      TOKENIZER.eos_token_id,
      TOKENIZER.convert_tokens_to_ids("<|eot_id|>")
  ]
  try:
    outputs = MODEL.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )
  except Exception as e:
      print(f"Error during model generation: {e}")
      exit()
  response_ids = outputs[0][input_ids.shape[-1]:]
  response_text = TOKENIZER.decode(response_ids, skip_special_tokens=True)

  # print("\nAI Assistant:")
  # print(response_text)
  return response_text



Using device: cuda


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
sessions = load_pickle("pseudo_session.pkl")
#print(sessions)
articles_df = pd.read_csv('articles_mapping.csv')
response_result = {}
for id, d in tqdm(sessions.items(),desc = "Generating response..."):
  start_time = time()
  N =30 - len(d['article_id'])
  system_prompt = load_prompt("system_prompt.txt")
  user_catorgory = articleID_to_product_type_name(articles_df,d['article_id'])
  user_query = f"User purchases {user_catorgory}, you need to predict the next {N} product categories."

  response = inference(system_prompt, user_query)


  print(f"{id} with N = {N}:{response}")
  response_result[id] = response
  print(f"Time taken: {time() - start_time:.2f} seconds")




Generating response...:   0%|          | 0/7 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Generating response...:  14%|█▍        | 1/7 [01:40<10:00, 100.05s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


1383:['Hair clip', 'Leggings/Tights', 'T-shirt', 'Socks', 'Underwear Tights', 'Sweater', 'Top', 'Trousers', 'Hat/beanie', 'Sneakers', 'Earring', 'Bag', 'Blazer', 'Other shoe', 'Pyjama set', 'Sandals', 'Jacket', 'Scarf', 'Coat']
Time taken: 100.05 seconds


Generating response...:  29%|██▊       | 2/7 [03:47<09:40, 116.00s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


1386:['Leggings/Tights', 'Hair clip', 'Socks', 'Underwear Tights', 'Leggings/Tights', 'Sweater', 'Top', 'Trousers', 'Hair string', 'Belt', 'Boots', 'Bikini top', 'Swimwear bottom', 'Underwear bottom', 'Swimsuit', 'Skirt', 'T-shirt', 'Dress', 'Hat/beanie', 'Sneakers', 'Sunglasses', 'Cardigan', 'Gloves', 'Earring', 'Bag', 'Blazer', 'Other shoe', 'Jumpsuit/Playsuit', 'Sandals']
Time taken: 127.17 seconds


Generating response...:  43%|████▎     | 3/7 [06:09<08:32, 128.01s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

8265:['Earrings', 'Bucket hat', 'Flat shoes', 'Vest top', 'T-shirt', 'Trousers', 'Socks', 'Hair clip', 'Hat/beanie', 'Scarf', 'Coat', 'Polo shirt', 'Blouse', 'Sandals', 'Jacket', 'Sunglasses', 'Cardigan', 'Gloves', 'Belt', 'Boots', 'Sneakers', 'Jumpsuit/Playsuit']
Time taken: 95.36 seconds


Generating response...:  86%|████████▌ | 6/7 [16:05<03:36, 216.26s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


8486:['Leggings/Tights', 'Socks', 'Hair clip', 'Umbrella', 'Pyjama jumpsuit/playsuit', 'Bodysuit', 'Hair string', 'Belt', 'Boots', 'Bikini top', 'Swimwear bottom', 'Underwear bottom', 'Swimsuit', 'Skirt', 'T-shirt', 'Dress', 'Hat/beanie', 'Kids Underwear top', 'Shorts', 'Shirt', 'Cap/peaked', 'Pyjama set', 'Sneakers', 'Sunglasses', 'Cardigan', 'Gloves', 'Earring', 'Bag', 'Blazer', 'Other shoe', 'Jumpsuit/Playsuit', 'Sandals', 'Jacket', 'Costumes', 'Robe', 'Scarf', 'Coat', 'Other accessories', 'Polo shirt', 'Slippers', 'Night gown', 'Alice band', 'Straw hat', 'Hat/brim', 'Tailored Waistcoat', 'Necklace', 'Ballerinas', 'Tie', 'Pyjama bottom', 'Felt hat', 'Bracelet', 'Blouse', 'Outdoor overall', 'Watch', 'Underwear body', 'Beanie', 'Giftbox', 'Sleeping sack', 'Dungarees', 'Outdoor trousers', 'Wallet', 'Swimwear set', 'Swimwear top', 'Flat shoe', 'Garment Set', 'Ring', 'Waterbottle', 'Wedge', 'Long John', 'Outdoor Waistcoat', 'Pumps', 'Flip flop', 'Braces', 'Bootie', 'Fine cosmetics', 'Hee

Generating response...: 100%|██████████| 7/7 [17:48<00:00, 152.59s/it]

11518:['Leggings/Tights', 'Hair clip', 'Trousers', 'Top', 'Belt', 'Boots', 'Skirt', 'T-shirt', 'Dress', 'Hat/beanie', 'Jumpsuit/Playsuit', 'Sandals', 'Jacket', 'Costumes', 'Robe', 'Scarf', 'Coat', 'Other accessories', 'Polo shirt', 'Slippers', 'Night gown', 'Alice band', 'Straw hat', 'Hat/brim']
Time taken: 102.19 seconds
save pickle  faild: [Errno 2] No such file or directory: ''


In [ ]:
save_pickle(response_result,"sessions_generation.pkl")


In [18]:
import ast
session_dict = load_pickle("sessions_generation.pkl")
for k,v in sessions.items():
  print(f"{k}: {len(v['article_id'])}\n{v}")

1383: 11
{'article_id': [25542, 31944, 35885, 2760, 15542, 15594, 44871, 25538, 17428, 28308, 26539], 't_dat': [Timestamp('2018-09-20 00:00:00'), Timestamp('2018-09-20 00:00:00'), Timestamp('2018-09-20 00:00:00'), Timestamp('2018-09-20 00:00:00'), Timestamp('2018-09-20 00:00:00'), Timestamp('2018-09-20 00:00:00'), Timestamp('2018-09-20 00:00:00'), Timestamp('2018-09-20 00:00:00'), Timestamp('2018-09-20 00:00:00'), Timestamp('2018-09-20 00:00:00'), Timestamp('2018-09-20 00:00:00')], 'price': [0.016932202503085136, 0.010152542032301426, 0.02540677972137928, 0.016932202503085136, 0.033881355077028275, 0.011847457848489285, 0.02540677972137928, 0.016932202503085136, 0.032186441123485565, 0.04828813672065735, 0.033881355077028275], 'sales_channel_id': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]}
1386: 7
{'article_id': [6641, 40723, 12618, 41391, 39645, 36492, 21375], 't_dat': [Timestamp('2018-09-20 00:00:00'), Timestamp('2018-09-20 00:00:00'), Timestamp('2018-10-07 00:00:00'), Timestamp('2018-10-07 0